In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/Users/a/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/a/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
PATH = '/Users/a/Desktop/lab/house_price/data/city24_export.csv'
EXPORT_TO = '/Users/a/Desktop/lab/house_price/data/city24_processed.csv'

In [3]:
df = pd.read_csv(PATH, ',')

In [4]:
def remove_last(string):
    if ':' in string:
        return string[:-1]
    return string

def replace_space(string):
    return string.replace(' ', '_')
df.columns = df.columns.map(str.lower).map(remove_last).map(replace_space)
cols_to_remove = ['first_image_link', 'description', 'otseviide', 
                  'lisainfo', 'kinnisvara_tüüp', 'lisaväärtused', 'ventilatsioon']
df.drop(cols_to_remove, axis=1, inplace=True)


In [5]:
df.drop(index=df[df['address'].isnull()].index, inplace=True)
df.dropna(subset=['hind','üldpind', 'tubade_arv'], inplace=True)
df.drop(['hind/ühik'], axis=1, inplace=True)
df.drop(['broker_name', 'lift'], axis=1, inplace=True)
df.dropna(subset=['broker_company'], inplace=True)


    


In [6]:
# divide address
df_address = pd.DataFrame(df.address.str.split(',',1).tolist(),columns = ['address','city_part'])    
df_address.count()
display(df_address['address'])

0                         Ao tn 2 - 34
1                              Kuninga
2                             Merirahu
3                       Ringi tn 60-26
4              Toom-Kuninga tn 21 - 34
5                       Taludevahe 116
6                              Rahumäe
7                Telliskivi tn 49 - 67
8                Telliskivi tn 49 - 28
9                           Ravi tn 14
10               Telliskivi tn 49 - 75
11                     Linnamäe tee 38
12                     Lutheri kvartal
13                            Lasnamäe
14                        Jakobi tn 15
15                Väike-Sepa tn 11 - 1
16                     Sõpruse pst 244
17                Telliskivi tn 49 - 5
18                     Linnamäe tee 12
19                    Pelguranna tn 23
20                       HEA PAKKUMINE
21                          Ümera tn 2
22                              Soprdi
23                               Ahtme
24       MÖBLEERITUD KORTER HEAS KOHAS
25                     Li

In [7]:
#divide linnaosad
kesklinn = df.loc[df['address'].str.contains('Kesklinna linnaosa')]
haabersti = df.loc[df['address'].str.contains('Haabersti linnaosa')]
kristiine = df.loc[df['address'].str.contains('Kristiine linnaosa')]
nomme = df.loc[df['address'].str.contains('Nõmme linnaosa')]
vanalinn = df.loc[df['address'].str.contains('Vanalinn')]
mustamae = df.loc[df['address'].str.contains('Mustamäe linnaosa')]
lasnamae = df.loc[df['address'].str.contains('Lasnamäe linnaosa')]
pirita = df.loc[df['address'].str.contains('Pirita linnaosa')]
pohja_tallinn = df.loc[df['address'].str.contains('Põhja-Tallinna linnaosa')]
kadriorg = df.loc[df['address'].str.contains('Kadriorg')]

frames = [kesklinn, haabersti, kristiine, nomme, vanalinn, mustamae, lasnamae, pirita, pohja_tallinn, kadriorg]
tln_df = pd.concat(frames)
tln_df.shape


(2185, 16)

In [8]:
#divide address column
def get_linnaosa(address):
    tln_parts = ['Kesklinn', 
                 'Haabersti', 
                 'Kristiine', 
                 'Nõmme', 
                 'Vanalinn', 
                 'Mustamäe', 
                 'Lasnamäe', 
                 'Pirita', 
                 'Põhja-Tallinn', 
                 'Kadriorg']
    for part in address:
        for word in tln_parts:
            if word in part:
                return word
            else:
                'Tallinn'
                            
tln_df['city_part'] = tln_df.address.str.split(',').apply(lambda x: get_linnaosa(x))

In [9]:
tln_df.shape

(2185, 17)

In [10]:
import re
pattern = re.compile("^[A-z-. ÄÖÜÕäöüõ]+\s+\d+")
def get_address(address):
    for part in address:
        part = part.rstrip().lstrip()
        match_obj = pattern.match(part)
        if match_obj:
            return match_obj.group()
    return np.nan

tln_df['address'] = tln_df.address.str.split(',').apply(lambda x: get_address(x))
tln_df.dropna(subset=['address'], inplace=True)


In [11]:
tln_df.count()


address              2148
hind                 2148
üldpind              2148
ehitusaasta          1763
hoone_materjal       1777
rõdu                 1217
küttesüsteem         1929
energiamärgis        1819
seisukord            1929
tubade_arv           2148
magamistubade_arv    1778
vannitubade_arv      1718
korrus/korruseid     2135
parkimine            1084
sanitaar             1737
broker_company       2148
city_part            2148
dtype: int64

In [12]:
# hind column cleaning from EUR
def clean_numeric_value(row):
    row = row.strip(' ')
    numeric = re.sub(r'\D', "", row)
    return numeric
tln_df['hind'] = tln_df['hind'].apply(lambda x: clean_numeric_value(x))

# change type of hind column
tln_df['hind'] = tln_df['hind'].astype(int)
tln_df.dtypes

address               object
hind                   int64
üldpind               object
ehitusaasta          float64
hoone_materjal        object
rõdu                  object
küttesüsteem          object
energiamärgis         object
seisukord             object
tubade_arv           float64
magamistubade_arv    float64
vannitubade_arv      float64
korrus/korruseid      object
parkimine             object
sanitaar              object
broker_company        object
city_part             object
dtype: object

In [13]:
#clean uldpind from m and convert to float
def clean_floating_value(row):
    row = row.strip(' ')
    floating = re.sub(r'[+-]?([0-9]*[.])?[0-9]+', "", row)
    return floating

tln_df['üldpind'] = tln_df['üldpind'].str.split(' ').apply(lambda x: x[0])
tln_df['üldpind'] = tln_df['üldpind'].astype(float)


In [14]:
# clean ehitusaasta column
tln_df.loc[tln_df['address'].str.contains('Pärnu mnt 69'),'ehitusaasta'] = 1911

In [15]:
tln_df[tln_df['ehitusaasta'].isnull()].shape

(366, 17)

In [16]:
# find same house objects that have ehitusaasta column aggregate the major value and assign
# null_year_address_list = tln_df[tln_df['ehitusaasta'].isnull()]['address'].tolist()
# for address in null_year_address_list:
#     year_isnotna = tln_df['ehitusaasta'].notnull()
#     has_addr = tln_df['address'] == address
#     found_year = tln_df[year_isnotna & has_addr]
#     if not found_year.empty:
#         found = found_year['ehitusaasta'].value_counts().index[0]
#         tln_df.loc[tln_df['address'] == address, 'ehitusaasta'] = found
# tln_df[tln_df['ehitusaasta'].isnull()].shape


In [17]:
def extract_column_from_similar_object(address_list_with_nan, nan_column, df):
    for address in address_list_with_nan:
        column_with_notna = df[nan_column].notnull()
        has_addr = df['address'] == address
        found_similar = df[column_with_notna & has_addr]
        if not found_similar.empty:
            found = found_similar[nan_column].value_counts().index[0]
            df.loc[df['address'] == address, nan_column] = found

In [18]:
# find same house objects that have ehitusaasta column aggregate the major value and assign
null_year_address_list = tln_df[tln_df['ehitusaasta'].isnull()]['address'].tolist()
extract_column_from_similar_object(null_year_address_list, 'ehitusaasta', tln_df)
tln_df[tln_df['ehitusaasta'].isnull()].shape

(276, 17)

In [19]:
# assign average ehitusaasta column value to nans
tallinn_average_build_year = tln_df[tln_df['ehitusaasta'].notnull()]['ehitusaasta'].mean()
display(tallinn_average_build_year)
tln_df.loc[tln_df['ehitusaasta'].isnull(), 'ehitusaasta'] = tallinn_average_build_year
tln_df[tln_df['ehitusaasta'].isnull()].shape

#change ehitusaasta column dtype
tln_df['ehitusaasta'] = tln_df['ehitusaasta'].astype(int)


1993.5106837606838

In [20]:
# drop hoone_material column because a lot of missing values
tln_df.drop(['hoone_materjal'], axis=1, inplace=True)
display(tln_df)

,address,hind,üldpind,ehitusaasta,rõdu,küttesüsteem,energiamärgis,seisukord,tubade_arv,magamistubade_arv,vannitubade_arv,korrus/korruseid,parkimine,sanitaar,broker_company,city_part
0,Ao tn 2,649800,148.3,2018,"klaasitud rõdu, rõdu, terrass","keskküte, põrandaküte",b,uusehitis,4.0,3.0,3.0,5 / 5,maaalune parkla,"saun, vann, wc ja vannituba eraldi, dušš",Perton Ehitus OÜ,Kesklinn
10,Ravi tn 14,190000,154.2,2004,prantsuse rõdu,elektriküte,d,remonti vajav,5.0,4.0,2.0,4 / 5,maaalune parkla,"saun, vann, tsentraalne kanalisatsioon, wc ja ...",DE Kodu OÜ,Kesklinn
13,Pärnu mnt 69,75000,14.9,1911,NaN,keskküte,b,renoveeritud,1.0,1.0,1.0,2 / 3,hoovis,"tsentraalne kanalisatsioon, dušš",Fisk OÜ,Kesklinn
15,Jakobi tn 15,77900,36.9,1960,NaN,keskküte,NaN,"renoveeritud, uus viimistlus, valmis",1.0,1.0,1.0,2 / 4,"tasuta parkimine, parkimiskoht kinnistul","tsentraalne kanalisatsioon, dušš",City Property OÜ,Kesklinn
33,Narva mnt 9,165000,76.0,1958,NaN,keskküte,Energiamärgis puudub,"renoveeritud, valmis",3.0,3.0,1.0,5 / 7,NaN,"vann, tsentraalne kanalisatsioon, wc ja vannit...",Eesti Kinnisvarabüroo OÜ,Kesklinn
35,Tartu mnt 52,209900,79.5,2017,klaasitud rõdu,põrandaküte,b,uusehitis,3.0,2.0,1.0,5 / 5,maaalune parkla,"vann, wc ja vannituba eraldi, dušš",Merko Ehitus AS,Kesklinn
38,Tina 13,125000,64.6,1963,rõdu,keskküte,d,san remont tehtud,3.0,2.0,1.0,1 / 5,NaN,NaN,RE Kinnisvara AS,Kesklinn
40,Pärnu mnt 69,119000,39.2,1911,NaN,keskküte,b,renoveeritud,2.0,1.0,1.0,2 / 3,hoovis,"tsentraalne kanalisatsioon, dušš",Fisk OÜ,Kesklinn
51,Vesivärava tn 50,269500,105.4,2019,rõdu,"keskküte, põrandaküte",b,uusehitis,4.0,3.0,2.0,9 / 6,NaN,"vann, wc ja vannituba eraldi",Baltic Sotheby's International Realty - Estonia,Kesklinn
52,Rävala pst 11,65000,25.9,1951,NaN,keskküte,d,NaN,1.0,1.0,1.0,1 / 5,NaN,"vann, pesumasin",RE Kinnisvara AS,Kesklinn


In [21]:
# extract terrass column because it may have influence on the price
tln_df['terrass'] = np.nan
tln_df.loc[tln_df['rõdu'].str.contains('terrass', na=False), 'terrass'] = 'terrass'
# remove old column and merge new
tln_df['terrass'] = pd.get_dummies(tln_df['terrass'])
tln_df.count()
# leave it now but probably later will 
# decide not to use it as it have small amount that will not improve model


address              2148
hind                 2148
üldpind              2148
ehitusaasta          2148
rõdu                 1217
küttesüsteem         1929
energiamärgis        1819
seisukord            1929
tubade_arv           2148
magamistubade_arv    1778
vannitubade_arv      1718
korrus/korruseid     2135
parkimine            1084
sanitaar             1737
broker_company       2148
city_part            2148
terrass              2148
dtype: int64

In [22]:
# clean rõdu column to have only true or false value indicating house has balcony or not
tln_df.loc[tln_df['rõdu'].str.contains('rõdu', na=False), 'rõdu'] = 'rõdu'
has_balcony_df = pd.get_dummies(tln_df['rõdu'])
has_balcony_df.drop(['terrass'], axis=1, inplace=True)
# remove old column and merge new
tln_df['rõdu'] = has_balcony_df
display(tln_df)

,address,hind,üldpind,ehitusaasta,rõdu,küttesüsteem,energiamärgis,seisukord,tubade_arv,magamistubade_arv,vannitubade_arv,korrus/korruseid,parkimine,sanitaar,broker_company,city_part,terrass
0,Ao tn 2,649800,148.3,2018,1,"keskküte, põrandaküte",b,uusehitis,4.0,3.0,3.0,5 / 5,maaalune parkla,"saun, vann, wc ja vannituba eraldi, dušš",Perton Ehitus OÜ,Kesklinn,1
10,Ravi tn 14,190000,154.2,2004,1,elektriküte,d,remonti vajav,5.0,4.0,2.0,4 / 5,maaalune parkla,"saun, vann, tsentraalne kanalisatsioon, wc ja ...",DE Kodu OÜ,Kesklinn,0
13,Pärnu mnt 69,75000,14.9,1911,0,keskküte,b,renoveeritud,1.0,1.0,1.0,2 / 3,hoovis,"tsentraalne kanalisatsioon, dušš",Fisk OÜ,Kesklinn,0
15,Jakobi tn 15,77900,36.9,1960,0,keskküte,NaN,"renoveeritud, uus viimistlus, valmis",1.0,1.0,1.0,2 / 4,"tasuta parkimine, parkimiskoht kinnistul","tsentraalne kanalisatsioon, dušš",City Property OÜ,Kesklinn,0
33,Narva mnt 9,165000,76.0,1958,0,keskküte,Energiamärgis puudub,"renoveeritud, valmis",3.0,3.0,1.0,5 / 7,NaN,"vann, tsentraalne kanalisatsioon, wc ja vannit...",Eesti Kinnisvarabüroo OÜ,Kesklinn,0
35,Tartu mnt 52,209900,79.5,2017,1,põrandaküte,b,uusehitis,3.0,2.0,1.0,5 / 5,maaalune parkla,"vann, wc ja vannituba eraldi, dušš",Merko Ehitus AS,Kesklinn,0
38,Tina 13,125000,64.6,1963,1,keskküte,d,san remont tehtud,3.0,2.0,1.0,1 / 5,NaN,NaN,RE Kinnisvara AS,Kesklinn,0
40,Pärnu mnt 69,119000,39.2,1911,0,keskküte,b,renoveeritud,2.0,1.0,1.0,2 / 3,hoovis,"tsentraalne kanalisatsioon, dušš",Fisk OÜ,Kesklinn,0
51,Vesivärava tn 50,269500,105.4,2019,1,"keskküte, põrandaküte",b,uusehitis,4.0,3.0,2.0,9 / 6,NaN,"vann, wc ja vannituba eraldi",Baltic Sotheby's International Realty - Estonia,Kesklinn,0
52,Rävala pst 11,65000,25.9,1951,0,keskküte,d,NaN,1.0,1.0,1.0,1 / 5,NaN,"vann, pesumasin",RE Kinnisvara AS,Kesklinn,0


In [23]:
tln_df.count()
tln_df.dtypes


address               object
hind                   int64
üldpind              float64
ehitusaasta            int64
rõdu                   uint8
küttesüsteem          object
energiamärgis         object
seisukord             object
tubade_arv           float64
magamistubade_arv    float64
vannitubade_arv      float64
korrus/korruseid      object
parkimine             object
sanitaar              object
broker_company        object
city_part             object
terrass                uint8
dtype: object

In [24]:
tln_df[tln_df['küttesüsteem'].isnull()].shape

(219, 17)

In [25]:
null_heating_address_list = tln_df[tln_df['küttesüsteem'].isnull()]['address'].tolist()
extract_column_from_similar_object(null_heating_address_list, 'küttesüsteem', tln_df)
tln_df[tln_df['küttesüsteem'].isnull()].shape


(152, 17)

In [26]:
most_popular_heating = tln_df['küttesüsteem'].value_counts().index[0]
tln_df['küttesüsteem'].fillna(most_popular_heating, inplace=True)
tln_df[tln_df['küttesüsteem'].isnull()].shape

(0, 17)

In [27]:
# extracting küttesüsteem column into categorical values
unique_heating_types = set()
def extract_unique(row, unique_set):
    for el in row.split(','):
        el = el.lstrip().rstrip()
        unique_set.add(el)
tln_df['küttesüsteem'].apply(lambda x: extract_unique(x, unique_heating_types))

for heating in unique_heating_types:
    tln_df[heating] = np.nan
    tln_df.loc[tln_df['küttesüsteem'].str.contains(heating), heating] = heating
    tln_df[heating] = pd.get_dummies(tln_df[heating])
tln_df.drop(['küttesüsteem'], axis=1, inplace=True)
tln_df.shape        


(2148, 27)

In [28]:
tln_df[tln_df['energiamärgis'].isnull()].shape
tln_df.dtypes

address                object
hind                    int64
üldpind               float64
ehitusaasta             int64
rõdu                    uint8
energiamärgis          object
seisukord              object
tubade_arv            float64
magamistubade_arv     float64
vannitubade_arv       float64
korrus/korruseid       object
parkimine              object
sanitaar               object
broker_company         object
city_part              object
terrass                 uint8
põrandaküte             uint8
gaasiküte               uint8
õhksoojuspump           uint8
kamin                   uint8
tahkekütus              uint8
kombineeritud küte      uint8
ahjuküte                uint8
autnoomne küte          uint8
maaküte                 uint8
elektriküte             uint8
keskküte                uint8
dtype: object

In [29]:
# processing energiamärgis column
null_energy_mark_address_list = tln_df[tln_df['energiamärgis'].isnull()]['address'].tolist()
extract_column_from_similar_object(null_energy_mark_address_list, 'energiamärgis', tln_df)

tln_df.loc[tln_df['energiamärgis'].isnull(), 'energiamärgis'] = 'b'
tln_df.loc[tln_df['energiamärgis'] == 'Energiamärgis puudub', 'energiamärgis'] = 'b'
energy_classes = pd.get_dummies(tln_df['energiamärgis'], prefix='energy_class')

tln_df = pd.concat([tln_df, energy_classes], axis=1)

tln_df.drop(['energiamärgis'], axis=1, inplace=True)

In [30]:
tln_df.columns
tln_df.dtypes

address                object
hind                    int64
üldpind               float64
ehitusaasta             int64
rõdu                    uint8
seisukord              object
tubade_arv            float64
magamistubade_arv     float64
vannitubade_arv       float64
korrus/korruseid       object
parkimine              object
sanitaar               object
broker_company         object
city_part              object
terrass                 uint8
põrandaküte             uint8
gaasiküte               uint8
õhksoojuspump           uint8
kamin                   uint8
tahkekütus              uint8
kombineeritud küte      uint8
ahjuküte                uint8
autnoomne küte          uint8
maaküte                 uint8
elektriküte             uint8
keskküte                uint8
energy_class_a          uint8
energy_class_b          uint8
energy_class_c          uint8
energy_class_d          uint8
energy_class_e          uint8
energy_class_f          uint8
energy_class_g          uint8
dtype: obj

In [31]:
tln_df[tln_df['seisukord'].isnull()].shape



(219, 33)

In [32]:
tln_df.loc[tln_df['ehitusaasta'] >= 2017, 'seisukord'] = 'uusehitis'
tln_df.loc[tln_df['seisukord'].isnull(), 'seisukord'] = 'valmis'
tln_df[tln_df['seisukord'].isnull()].shape


(0, 33)

In [33]:
unique_house_conditions = set()
tln_df['seisukord'].apply(lambda x: extract_unique(x, unique_house_conditions))
unique_house_conditions


{'kap. remonti vajav',
 'remonti vajav',
 'renoveeritud',
 'san remont tehtud',
 'uus viimistlus',
 'uusehitis',
 'valmimisjärgus',
 'valmis'}

In [34]:
# i decide to estimate condition by using next classes [A,B,C]
# A - new
# B - ready
# C - requires fixes
# make dictionary with unique_house_conditions
house_condition = {'A':['uusehitis'], 'B':['uus viimistlus', 'valmis', 'san remont tehtud', 'renoveeritud'], 'C':['remonti vajav', 'valmimisjärgus', 'kap. remonti vajav']}
def estimate_house_condition(row):
    weights = {'A':3, 'B':2, 'C':1}
    resulting_estimation = 'A'
    for mark, conditions in house_condition.items():
        for condition in conditions:
            if condition in row:
                if weights.get(resulting_estimation) > weights.get(mark):
                    resulting_estimation = mark
    return resulting_estimation
tln_df['seisukord'] = tln_df['seisukord'].apply(lambda x: estimate_house_condition(x))
display(tln_df)
            

,address,hind,üldpind,ehitusaasta,rõdu,seisukord,tubade_arv,magamistubade_arv,vannitubade_arv,korrus/korruseid,...,maaküte,elektriküte,keskküte,energy_class_a,energy_class_b,energy_class_c,energy_class_d,energy_class_e,energy_class_f,energy_class_g
0,Ao tn 2,649800,148.3,2018,1,A,4.0,3.0,3.0,5 / 5,...,0,0,1,0,1,0,0,0,0,0
10,Ravi tn 14,190000,154.2,2004,1,C,5.0,4.0,2.0,4 / 5,...,0,1,0,0,0,0,1,0,0,0
13,Pärnu mnt 69,75000,14.9,1911,0,B,1.0,1.0,1.0,2 / 3,...,0,0,1,0,1,0,0,0,0,0
15,Jakobi tn 15,77900,36.9,1960,0,B,1.0,1.0,1.0,2 / 4,...,0,0,1,0,0,0,0,0,1,0
33,Narva mnt 9,165000,76.0,1958,0,B,3.0,3.0,1.0,5 / 7,...,0,0,1,0,1,0,0,0,0,0
35,Tartu mnt 52,209900,79.5,2017,1,A,3.0,2.0,1.0,5 / 5,...,0,0,0,0,1,0,0,0,0,0
38,Tina 13,125000,64.6,1963,1,B,3.0,2.0,1.0,1 / 5,...,0,0,1,0,0,0,1,0,0,0
40,Pärnu mnt 69,119000,39.2,1911,0,B,2.0,1.0,1.0,2 / 3,...,0,0,1,0,1,0,0,0,0,0
51,Vesivärava tn 50,269500,105.4,2019,1,A,4.0,3.0,2.0,9 / 6,...,0,0,1,0,1,0,0,0,0,0
52,Rävala pst 11,65000,25.9,1951,0,B,1.0,1.0,1.0,1 / 5,...,0,0,1,0,0,0,1,0,0,0


In [35]:
# decided to remove 'magamistubade_arv', 'vannitubade_arv' columns
# as they does not improve model much
tln_df['tubade_arv'] = tln_df['tubade_arv'].astype(int)
tln_df.drop(['magamistubade_arv', 'vannitubade_arv'], axis=1, inplace=True)


In [36]:
tln_df.count()

address               2148
hind                  2148
üldpind               2148
ehitusaasta           2148
rõdu                  2148
seisukord             2148
tubade_arv            2148
korrus/korruseid      2135
parkimine             1084
sanitaar              1737
broker_company        2148
city_part             2148
terrass               2148
põrandaküte           2148
gaasiküte             2148
õhksoojuspump         2148
kamin                 2148
tahkekütus            2148
kombineeritud küte    2148
ahjuküte              2148
autnoomne küte        2148
maaküte               2148
elektriküte           2148
keskküte              2148
energy_class_a        2148
energy_class_b        2148
energy_class_c        2148
energy_class_d        2148
energy_class_e        2148
energy_class_f        2148
energy_class_g        2148
dtype: int64

In [37]:
null_house_floor_address_list = tln_df[tln_df['korrus/korruseid'].isnull()]['address'].tolist()
extract_column_from_similar_object(null_house_floor_address_list, 'korrus/korruseid', tln_df)
tln_df['korrus/korruseid'].fillna(tln_df['korrus/korruseid'].value_counts().index[0], inplace=True)


In [38]:
# extract korrus from korrus/korruseid column
tln_df['korrus/korruseid'] = tln_df['korrus/korruseid'].str.replace('-','/')
tln_df['korrus'] = tln_df['korrus/korruseid'].str.strip().str[0]
tln_df.drop(['korrus/korruseid'], axis=1, inplace=True)


In [39]:
# remove parkimine column because half of the values are missed
tln_df.drop(['parkimine'], axis=1, inplace=True)

In [40]:
# extract specific features like pool, sauna and etc..
tln_df.loc[tln_df['sanitaar'].isnull(), 'sanitaar'] = 'dušš'
tln_df['saun'] = np.nan
has_saun = tln_df['sanitaar'].str.contains('saun')
tln_df.loc[has_saun, 'saun'] = 'saun'
tln_df['saun'] = pd.get_dummies(tln_df['saun'])

tln_df['bassein'] = np.nan
tln_df.loc[tln_df['sanitaar'].str.contains('bassein'), 'bassein'] = 'bassein'
tln_df['bassein'] = pd.get_dummies(tln_df['bassein'])

tln_df.drop(['sanitaar'], axis=1, inplace=True)

In [41]:
tln_df.columns = ['address', 'hind', 'üldpind', 'ehitusaasta', 'rõdu', 'seisukord',
       'tubade_arv', 'broker', 'linnaosa', 'terrass', 'maaküte',
       'autnoomne küte', 'gaasiküte', 'kombineeritud küte', 'kamin',
       'õhksoojuspump', 'põrandaküte', 'tahkekütus', 'keskküte', 'ahjuküte',
       'elektriküte', 'energy_class_a', 'energy_class_b', 'energy_class_c',
       'energy_class_d', 'energy_class_e', 'energy_class_f', 'energy_class_g',
       'korrus', 'saun', 'bassein']
tln_df.to_csv(EXPORT_TO, encoding='utf-8')
tln_df


,address,hind,üldpind,ehitusaasta,rõdu,seisukord,tubade_arv,broker,linnaosa,terrass,...,energy_class_a,energy_class_b,energy_class_c,energy_class_d,energy_class_e,energy_class_f,energy_class_g,korrus,saun,bassein
0,Ao tn 2,649800,148.3,2018,1,A,4,Perton Ehitus OÜ,Kesklinn,1,...,0,1,0,0,0,0,0,5,1,0
10,Ravi tn 14,190000,154.2,2004,1,C,5,DE Kodu OÜ,Kesklinn,0,...,0,0,0,1,0,0,0,4,1,0
13,Pärnu mnt 69,75000,14.9,1911,0,B,1,Fisk OÜ,Kesklinn,0,...,0,1,0,0,0,0,0,2,0,0
15,Jakobi tn 15,77900,36.9,1960,0,B,1,City Property OÜ,Kesklinn,0,...,0,0,0,0,0,1,0,2,0,0
33,Narva mnt 9,165000,76.0,1958,0,B,3,Eesti Kinnisvarabüroo OÜ,Kesklinn,0,...,0,1,0,0,0,0,0,5,0,0
35,Tartu mnt 52,209900,79.5,2017,1,A,3,Merko Ehitus AS,Kesklinn,0,...,0,1,0,0,0,0,0,5,0,0
38,Tina 13,125000,64.6,1963,1,B,3,RE Kinnisvara AS,Kesklinn,0,...,0,0,0,1,0,0,0,1,0,0
40,Pärnu mnt 69,119000,39.2,1911,0,B,2,Fisk OÜ,Kesklinn,0,...,0,1,0,0,0,0,0,2,0,0
51,Vesivärava tn 50,269500,105.4,2019,1,A,4,Baltic Sotheby's International Realty - Estonia,Kesklinn,0,...,0,1,0,0,0,0,0,9,0,0
52,Rävala pst 11,65000,25.9,1951,0,B,1,RE Kinnisvara AS,Kesklinn,0,...,0,0,0,1,0,0,0,1,0,0
